In [19]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import time
import openpyxl
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait


premium_df = pd.DataFrame(columns = ['성별','생년월일','보험 기간', '납입 기간', '납입 주기',
                                    '주 보험 금액', '특약 이름', '특약 금액', '보험료'])

driver = webdriver.Chrome("C:\\Users\\dngus\\dev\\2019Summer\\CONCAT\\LinaCrwaling\\chromdriver_win32\\chromedriver.exe")

url = "https://www.lina.co.kr/disclosure/insr_price.htm"
driver.get(url)

page = driver.page_source
soup = BeautifulSoup(page, 'html.parser')


# 보험료 계산 페이지 접근하기
table = soup.select("#content > table:nth-child(30)") # 치아 보험들이 있는 div
links = table[0].find_all("a")

premium_names = table[0].find_all("td",class_= "bdl")
premium_name  = premium_names[0].get_text()

for i  in range(0,len(links)):
    links[i] = links[i].get('href')
    links[i] = links[i][1:]

# 보험료 계산하기
baseUrl = "https://www.lina.co.kr/product/simulation.htm?paramProductCode="
productUrl = baseUrl + links[0]

driver.get(productUrl)

# 이름 넣기
driver.find_element_by_name('name').send_keys('홍길동')

wait = WebDriverWait(driver,5)

loc_number = 0

# 보험 가능 나이 계산하기
age_regex = re.compile(r'[0-9]+')
age_range = driver.find_element_by_class_name('tbl_txt_01').text
age_      = age_regex.findall(age_range)

old_age   = 2019 - int(age_[1]) # 가입 가능한 가장 늙은 나이
young_age = 2019 - int(age_[0]) # 가입 가능한 가장 어린 나이

for age_i in range(0,100):
    if age_i % 10 == 0:
        time.sleep(2)
    driver.find_element_by_name('iresid_no1').clear()
    time.sleep(0.1)
    
    # 나이 선택하기
    
    age_year = old_age + age_i
    age_m_d  ='0625'
    age = str(age_year) + age_m_d
    
    if age_year > young_age :
        break
    

    driver.find_element_by_name('iresid_no1').send_keys(age)
    time.sleep(0.1)

    # 성별 선택하기
    for sex_i in range(2):
        sex = '남성'
        

        if sex_i == 1:
            # 여성 선택
            sex_radio_btn = wait.until(EC.element_to_be_clickable((By.ID,'leftinlabel1_woman')))
            sex_radio_btn.click()
            #driver.find_element_by_id('leftinlabel1_woman').click()
            sex = '여성'
        else :
            sex_radio_btn = wait.until(EC.element_to_be_clickable((By.ID,'leftinlabel1_man')))
            sex_radio_btn.click()
            #driver.find_element_by_id('leftinlabel1_man').click()
            sex = '남성'
        
            
            
        time.sleep(0.7)

        # 보험 기간 선택하기
        policy_period  = driver.find_element_by_id('policy_period')
        policy_options = policy_period.find_elements_by_tag_name('option')

        for policy_option in policy_options:
            if policy_option.text == '선택':
                continue
            
            policy_option.click()
            policy_option_name = policy_option.text
            time.sleep(0.1)


            # 납입 기간 선택하기
            pay_period  = driver.find_element_by_id('pay_period')
            pay_options = pay_period.find_elements_by_tag_name('option')

            for pay_option in pay_options:
                if pay_option.text == '선택':
                    continue

                pay_option.click()
                pay_option_name = pay_option.text
                time.sleep(0.1)


                # 납입 주기 선택하기
                premium_mode = driver.find_element_by_name('premium_mode')
                pm_options   = premium_mode.find_elements_by_tag_name('option')

                for pm_option in pm_options:
                    if pm_option.text == '선택':
                        continue

                    pm_option.click()
                    pm_option_name = pm_option.text
                    time.sleep(0.1)



                    # 주 보험 가입 금액 선택
                    product_amount = driver.find_element_by_name('product_amount')
                    pa_options     = product_amount.find_elements_by_tag_name('option')

                    for pa_option in pa_options:
                        if pa_option.text == '선택':
                            continue

                        pa_option.click()
                        pa_option_name = pa_option.text
                        time.sleep(0.1)


                        # 특약

                        # 특약 정보를 담은 테이블
                        rider_table = driver.find_element_by_id('riderInfo_div')
                        rider_list  = rider_table.find_elements_by_tag_name('tbody')[0].find_elements_by_tag_name('tr')

                        for rider in rider_list:
                            rider_amount  = rider.find_element_by_name('rider_amount')
                            rider_options = rider_amount.find_elements_by_tag_name('option')
                            rider_name    = rider.find_element_by_id('rider_name').get_attribute("value")

                            for rider_option in rider_options:
                                if rider_option.text == '선택':
                                    continue

                                rider_option.click()
                                rider_option_name = rider_option.text
                                time.sleep(0.1)


                                # 보험료 조회하기
                                
                                calc_btn = wait.until(EC.element_to_be_clickable((By.CLASS_NAME,'g_btn_09.btnProductPremium')))
                                calc_btn.click()
                                #driver.find_element_by_class_name("g_btn_09.btnProductPremium").click()
                                time.sleep(0.5)

                                result = driver.find_element_by_id("prm_result").get_attribute("value")
                                
                                print(result)

                                result = re.sub(r'[A-Z]+[0-9]+:','',result)
                                print("result : ", result)

                                regex   = re.compile(r'[0-9]+')
                                premium = regex.findall(result)
                                print(premium)
                                print("몇번? : ",age_i)
                                premium = int(premium[0]) + int(premium[1])

                                premium_df.loc[loc_number] = [sex, age, policy_option_name,pay_option_name, pm_option_name, pa_option_name,
                                                         rider_name, rider_option_name, premium]
                                loc_number += 1
                                time.sleep(2)

writer = pd.ExcelWriter(premium_name + ".xlsx")
premium_df.to_excel(writer,'Sheet1')
writer.save()


B00176005:18818,R00153004:16247
result :  18818,16247
['18818', '16247']
몇번? :  0
B00176005:18818,R00153004:32495
result :  18818,32495
['18818', '32495']
몇번? :  0
B00176005:20952,R00153004:11446
result :  20952,11446
['20952', '11446']
몇번? :  0
B00176005:20952,R00153004:22892
result :  20952,22892
['20952', '22892']
몇번? :  0
B00176005:18527,R00153004:15859
result :  18527,15859
['18527', '15859']
몇번? :  1
B00176005:18527,R00153004:15859
result :  18527,15859
['18527', '15859']
몇번? :  1
B00176005:20564,R00153004:10961
result :  20564,10961
['20564', '10961']
몇번? :  1
B00176005:20564,R00153004:21922
result :  20564,21922
['20564', '21922']
몇번? :  1
B00176005:18333,R00153004:15326
result :  18333,15326
['18333', '15326']
몇번? :  2
B00176005:18333,R00153004:30652
result :  18333,30652
['18333', '30652']
몇번? :  2
B00176005:20079,R00153004:10427
result :  20079,10427
['20079', '10427']
몇번? :  2
B00176005:20079,R00153004:20855
result :  20079,20855
['20079', '20855']
몇번? :  2
B00176005:18042,

B00176005:18042,R00153004:3298
result :  18042,3298
['18042', '3298']
몇번? :  25
B00176005:18042,R00153004:6596
result :  18042,6596
['18042', '6596']
몇번? :  25
B00176005:16490,R00153004:3637
result :  16490,3637
['16490', '3637']
몇번? :  26
B00176005:16490,R00153004:7275
result :  16490,7275
['16490', '7275']
몇번? :  26
B00176005:17945,R00153004:3201
result :  17945,3201
['17945', '3201']
몇번? :  26
B00176005:17945,R00153004:6402
result :  17945,6402
['17945', '6402']
몇번? :  26
B00176005:16199,R00153004:3346
result :  16199,3346
['16199', '3346']
몇번? :  27
B00176005:16199,R00153004:6693
result :  16199,6693
['16199', '6693']
몇번? :  27
B00176005:17848,R00153004:3104
result :  17848,3104
['17848', '3104']
몇번? :  27
B00176005:17848,R00153004:6208
result :  17848,6208
['17848', '6208']
몇번? :  27
B00176005:17848,R00153004:6208
result :  17848,6208
['17848', '6208']
몇번? :  28
B00176005:15908,R00153004:6208
result :  15908,6208
['15908', '6208']
몇번? :  28
B00176005:17751,R00153004:3007
result : 

In [4]:
result = driver.find_element_by_id("prm_result").get_attribute("value")

In [5]:
result

'B00176005:16587,R00153004:9312'

In [17]:
wait.until(EC.element_to_be_clickable((By.CLASS_NAME,'g_btn_09.btnProductPremium'))).click()